In [60]:
import pandas as pd
df= pd.read_csv("data\parsed.csv")
print(df.head())

  alert  cdi      code                                             detail  \
0   NaN  NaN  37389218  https://earthquake.usgs.gov/fdsnws/event/1/que...   
1   NaN  NaN  37389202  https://earthquake.usgs.gov/fdsnws/event/1/que...   
2   NaN  4.4  37389194  https://earthquake.usgs.gov/fdsnws/event/1/que...   
3   NaN  NaN  37389186  https://earthquake.usgs.gov/fdsnws/event/1/que...   
4   NaN  NaN  73096941  https://earthquake.usgs.gov/fdsnws/event/1/que...   

       dmin  felt    gap           ids   mag magType  ...     status  \
0  0.008693   NaN   85.0  ,ci37389218,  1.35      ml  ...  automatic   
1  0.020030   NaN   79.0  ,ci37389202,  1.29      ml  ...  automatic   
2  0.021370  28.0   21.0  ,ci37389194,  3.42      ml  ...  automatic   
3  0.026180   NaN   39.0  ,ci37389186,  0.44      ml  ...  automatic   
4  0.077990   NaN  192.0  ,nc73096941,  2.16      md  ...  automatic   

            time                          title tsunami        type  \
0  1539475168010  M 1.4 - 9km NE 

In [20]:
#1. mb 지진 규모 유형으로 사용해 일본 진도의 95번째 백분위수를 찾는다.

mt= df[(df.magType == 'mb') &  (df.parsead_place=='Japan')]
mt_95 = mt['mag'].quantile(0.95)
print(mt_95)

4.9


In [49]:
#2. 인도네시아에서 쓰나미가 동반된 지진의 백분율을 구한다.

q2 = len(df[df.parsed_place=='Indonesia'])
q2_s =  len(df[(df.parsed_place=='Indonesia') & (df.tsunami==1)])
answer = (q2_s/q2) *100
print(answer)

23.12925170068027


In [54]:
#3. 네바다 지진에 대한 요약통계를 계산한다.

df[df.parsed_place=='Nevada'].describe()

,cdi,dmin,felt,gap,mag,mmi,nst,rms,sig,time,tsunami,tz,updated
count,15.000000,681.000000,15.000000,681.000000,681.000000,1.00,681.000000,681.000000,681.000000,6.810000e+02,681.0,681.0,6.810000e+02
mean,2.440000,0.166199,2.400000,153.668120,0.500073,2.84,12.618209,0.151986,10.970631,1.538314e+12,0.0,-480.0,1.538402e+12
std,0.501142,0.166228,4.626013,68.735302,0.696710,NaN,9.866963,0.084662,19.607150,5.965637e+08,0.0,0.0,6.010951e+08
min,2.000000,0.001000,1.000000,29.140000,-0.500000,2.84,3.000000,0.000500,0.000000,1.537247e+12,0.0,-480.0,1.537307e+12
25%,2.000000,0.053000,1.000000,97.380000,-0.100000,2.84,6.000000,0.106900,0.000000,1.537854e+12,0.0,-480.0,1.537928e+12
50%,2.200000,0.112000,1.000000,149.140000,0.400000,2.84,10.000000,0.146300,2.000000,1.538280e+12,0.0,-480.0,1.538428e+12
75%,2.900000,0.233000,1.000000,199.720000,0.900000,2.84,16.000000,0.187100,12.000000,1.538821e+12,0.0,-480.0,1.538878e+12
max,3.300000,1.414000,19.000000,355.910000,2.900000,2.84,61.000000,0.863400,129.000000,1.539461e+12,0.0,-480.0,1.539483e+12


In [88]:
#4. 지진이 불의 고리에 있는 국가나 미국 주에서 발생했는지를 나타내는 열을 추가 한다.
# 알래스카Aaska, 남극 대륙 Antaratica, Antarctic, 볼리비아 Boliva, 캘리포니아 callornie, 캐나다 Canada, 
# 칠레Chile, 코스타리기costa. Rica, 에콰도르 Eouador, 피 지대, 과테 말라Guatemala. 인도네시아 Indonesia. 일본Japan 
# 케르마덱 제도Kermadec Inslands. 멕시코 MexO0 (뉴멕시코New MeXiCO를 선택하면 안 된다), 뉴질랜드New zealand, 
# 페루Pen., 필리핀 Philppins, 러시아 Ausla, 타이완'awan, 통가Tonga, 워싱턴Washington 을 사용한다.

df['ring_of_fire'] = df.parsed_place.str.contains(r'|'.join([ # r ='l' =>  or연산자를 사용하기위해 앞에 넣어줌 
    'Alaska', 'Antarctic', 'Bolivia', 'California', 'Canada',
    'Chile', 'Costa Rica', 'Ecuador', 'Fiji', 'Guatemala',
    'Indonesia', 'Japan', 'Kermadec Islands', '^Mexico',
    'New Zealand', 'Peru', 'Philippines', 'Russia',
    'Taiwan', 'Tonga', 'Washington' 
]))

print(df['ring_of_fire'])

0        True
1        True
2        True
3        True
4        True
        ...  
9327     True
9328     True
9329    False
9330     True
9331     True
Name: ring_of_fire, Length: 9332, dtype: bool


In [89]:
#5. 불의 고리 위치에서 발생한 지진의 횟수와 그 외에서 발생한 지진의 횟수를 계산한다.

df['ring_of_fire'].value_counts()


True     7188
False    2144
Name: ring_of_fire, dtype: int64

In [103]:
# 불의 고리를 따라 발생한 쓰나미의 횟수 

df_6 = df[(df['ring_of_fire']==True) & (df['tsunami']==1)]
print(len(df_6))

45
